# Proyecto Final Lab Inteligencia (Imagenes): Método alternativo 

In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.spatial import distance as dist
import argparse
import glob

## 7) Uso de un método alternativo para la extracción de características

Se utiliza el método HoG (Histograms of Gradients) para realizar la extracción de caractrísticas. Se reutilizan varias de las funciones utilizadas anteriormente.

### 7.1) Lectura de imagen (en RGB)

In [2]:
#Funcion que lear una imagen a RGB
def leerRGB (nombreImagen):
    #Lectura
    readImage = cv2.imread(nombreImagen)
    return readImage

In [7]:
test = leerRGB('100501.jpg')

In [8]:
cv2.imshow('image',test)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
#Dimensiones de cada una de las imagenes
test.shape

(2448, 3264, 3)

### 7.2) División de la imagen en razones 1:2 (6filasx3columas)

In [23]:
#Funcion que recorte la imagen en 6x3 celdas. Devuelve 18 submatrices.
def div9Celdas(imRGB):
    #division
    R = imRGB[:,:,0]
    G = imRGB[:,:,1]
    B = imRGB[:,:,2]
    #Dimensiones de la imagen
    alto,ancho,prof = imRGB.shape
    #Nuevas dimensiones
    nuevoAlto = int(alto/6)
    nuevoAncho = int(ancho/3)
    #Creacion de las tres matrices (inicialmente vacias).
    Rdiv = np.zeros([nuevoAlto,nuevoAncho,18])
    Gdiv = np.zeros([nuevoAlto,nuevoAncho,18])
    Bdiv = np.zeros([nuevoAlto,nuevoAncho,18])
    #Dividir imagen
    vecDivisionesAlto = [0,nuevoAlto,nuevoAlto,nuevoAlto*2,nuevoAlto*2,nuevoAlto*3,nuevoAlto*3,nuevoAlto*4,nuevoAlto*4,nuevoAlto*5,nuevoAlto*5,nuevoAlto*6]
    vecDivisionesAncho = [0,nuevoAncho,nuevoAncho,nuevoAncho*2,nuevoAncho*2,nuevoAncho*3]
    count = 0
    count1 = 0
    while count1 <= 11:
        count2 = 0
        while count2 <= 5:
            Rdiv[:,:,count] = R[vecDivisionesAlto[count1]:vecDivisionesAlto[count1+1],vecDivisionesAncho[count2]:vecDivisionesAncho[count2+1]]
            Gdiv[:,:,count] = G[vecDivisionesAlto[count1]:vecDivisionesAlto[count1+1],vecDivisionesAncho[count2]:vecDivisionesAncho[count2+1]]
            Bdiv[:,:,count] = B[vecDivisionesAlto[count1]:vecDivisionesAlto[count1+1],vecDivisionesAncho[count2]:vecDivisionesAncho[count2+1]]
            count += 1
            count2 += 2
        count1 +=2  
        
    return np.uint8(Rdiv),np.uint8(Gdiv),np.uint8(Bdiv)

### 7.3) Cálculo de los gradientes.

In [27]:
#Funcion que calcule los gradientes de las matrices RGB en x e y.
def gradienteCartesianas(imRGB):
    #Se almacenan los valores en tres matrices, correspondientes a R,G y B.
    #Division de la imagen.
    Rdiv,Gdiv,Bdiv = div9Celdas(imRGB)
    #Calculo del gradiente en x e y.
    #R.
    gxR = cv2.Sobel(Rdiv, cv2.CV_32F, 1, 0, ksize=1)
    gyR = cv2.Sobel(Rdiv, cv2.CV_32F, 0, 1, ksize=1)
    #G
    gxG = cv2.Sobel(Gdiv, cv2.CV_32F, 1, 0, ksize=1)
    gyG = cv2.Sobel(Gdiv, cv2.CV_32F, 0, 1, ksize=1)
    #B
    gxB = cv2.Sobel(Bdiv, cv2.CV_32F, 1, 0, ksize=1)
    gyB = cv2.Sobel(Bdiv, cv2.CV_32F, 0, 1, ksize=1)
    return gxR,gyR,gxG,gyG,gxB,gyB

#Funcion que calcule las matrices de magnitud y orientación
def gradientePolares(imRGB):
    #Calculo del gradiente en polares.
    gxR,gyR,gxG,gyG,gxB,gyB = gradienteCartesianas(imRGB)
    #Paso a coordenadas polares.
    #R
    magR, angleR = cv2.cartToPolar(gxR, gyR, angleInDegrees=True)
    #G
    magG, angleG = cv2.cartToPolar(gxG, gyG, angleInDegrees=True)
    #B
    magB, angleB = cv2.cartToPolar(gxB, gyB, angleInDegrees=True)
    return magR,angleR,magG,angleG,magB,angleB

#Lo que se obtienen son matrices de dimensiones 408 por 1088 por 18.

### 7.4) Cálculo de los histogramas.

In [38]:
#Funcion que normalice histograma.
#Funcion que realice la normalización de histogramas
def normHist(histogram):
    sumaHist = np.sum(histogram)
    normHist = (1/sumaHist)*histogram
    normHist = np.ravel(normHist)
    return normHist

#Funcion que cree un histograma para una matriz de magnitudes y una matriz de angulo.
def creacionHistogramaIndividual(mag,angle):
    #Dimensiones de las divisiones
    altura,ancho,prof = mag.shape
    #Creación del arreglo histograma
    hist = np.array([0,0,0,0,0,0,0,0,0])
    #Generación histograma
    count1 = 0
    count2 = 0
    while count1 < altura:
        while count2 < ancho:
            #Casos de igualdad.
            if(angle[count1,count2]==0):
                hist[0] = hist[0] + mag[count1,count2]
            elif(angle[count1,count2]==20):
                hist[1] = hist[1] + mag[count1,count2]
            elif(angle[count1,count2]==40):
                hist[2] = hist[2] + mag[count1,count2]
            elif(angle[count1,count2]==60):
                hist[3] = hist[3] + mag[count1,count2]
            elif(angle[count1,count2]==80):
                hist[4] = hist[4] + mag[count1,count2]
            elif(angle[count1,count2]==100):
                hist[5] = hist[5] + mag[count1,count2]
            elif(angle[count1,count2]==120):
                hist[6] = hist[6] + mag[count1,count2]
            elif(angle[count1,count2]==140):
                hist[7] = hist[7] + mag[count1,count2]
            elif(angle[count1,count2]==160):
                hist[8] = hist[8] + mag[count1,count2]                
            #Casos de valores intermedios.
            elif((angle[count1,count2]>0) and (angle[count1,count2]<20)):
                hist[0] = hist[0] + mag[count1,count2]/2
                hist[1] = hist[1] + mag[count1,count2]/2
            elif((angle[count1,count2]>20) and (angle[count1,count2]<40)):
                hist[1] = hist[1] + mag[count1,count2]/2
                hist[2] = hist[2] + mag[count1,count2]/2
            elif((angle[count1,count2]>40) and (angle[count1,count2]<60)):
                hist[2] = hist[2] + mag[count1,count2]/2
                hist[3] = hist[3] + mag[count1,count2]/2
            elif((angle[count1,count2]>60) and (angle[count1,count2]<80)):
                hist[3] = hist[3] + mag[count1,count2]/2
                hist[4] = hist[4] + mag[count1,count2]/2
            elif((angle[count1,count2]>80) and (angle[count1,count2]<100)):
                hist[4] = hist[4] + mag[count1,count2]/2
                hist[5] = hist[5] + mag[count1,count2]/2
            elif((angle[count1,count2]>100) and (angle[count1,count2]<120)):
                hist[5] = hist[5] + mag[count1,count2]/2
                hist[6] = hist[6] + mag[count1,count2]/2
            elif((angle[count1,count2]>120) and (angle[count1,count2]<140)):
                hist[6] = hist[6] + mag[count1,count2]/2
                hist[7] = hist[7] + mag[count1,count2]/2
            elif((angle[count1,count2]>140) and (angle[count1,count2]<160)):
                hist[7] = hist[7] + mag[count1,count2]/2
                hist[8] = hist[8] + mag[count1,count2]/2
            elif((angle[count1,count2]>160) and (angle[count1,count2]<180)):
                hist[8] = hist[8] + mag[count1,count2]/2
                hist[0] = hist[0] + mag[count1,count2]/2
            count2 += 1
        count1 += 1
    
    histNormalizado = normHist(hist)
    return histNormalizado

#Funcion que concatene los histogramas de las divisiones para generar el histograma de canal.
def creacionHistogramaCanal(magCanal,angCanal):
    #Dimensiones.
    alto,ancho,prof = magCanal.shape
    #Creacion de arreglo del histograma conjunto
    histConjunto = np.zeros(prof*9)
    #Iteraciones sobre las distintas divisiones.
    count = 0
    while count < prof:
        histDivision = creacionHistogramaIndividual(magCanal[count],angCanal[count])
        histConjunto[count*9:count*9+9] = histDivision
        count += 1
    return histConjunto      
      
#Funcion que genere el histograma concatenado para una imagen.
def histogramaGradienteImagen(imRGB):
    #Calculo de los gradientes en polares.
    magR,angleR,magG,angleG,magB,angleB = gradientePolares(imRGB)
    #Calculo de los histogramas:
    histR = creacionHistogramaCanal(magR,angleR)
    histG = creacionHistogramaCanal(magG,angleG)
    histB = creacionHistogramaCanal(magB,angleB)
    #Concatenacion
    hisTotal = np.concatenate((histR,histG,histB))
    #Resultado
    return histTotal
    

In [28]:
True and False

False

In [31]:
a = np.zeros(9)
b = np.ones(9)

In [37]:
np.concatenate((a,b,a))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])